<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/3_18%ED%8C%8C%EC%9D%B4%EC%8D%AC(matplotlib%2C%EC%8B%9C%EA%B0%81%ED%99%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
[문제125] 2003년 이전의 입사한 사원들의 정보를 출력하세요.
from pandas import Series, DataFrame
import pandas as pd

emp = pd.read_csv('c:/data/employees.csv')

emp[emp['HIRE_DATE'] < '2003-01-01'][['EMPLOYEE_ID','HIRE_DATE']]
emp[pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d') < pd.to_datetime('2003-01-01')]
emp[pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.year < 2003]

pd.Timestamp.now().year
pd.Timestamp.now().dayofweek
pd.Timestamp.now().weekday()

pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d')[1].year
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.year # year만 뽑고 싶을 때 dt 사용
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.month
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.day
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.hour
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.minute
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.second
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.microsecond
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.dayofweek
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.weekday
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.day_name()

[문제126] 2005년도에 입사한 사원들의 급여의 총액을 출력하세요.
emp[pd.to_datetime(emp['HIRE_DATE']).dt.year == 2005][['SALARY']].sum()

[문제127] 년도별 총액급여를 출력하세요.
emp.SALARY.groupby(pd.to_datetime(emp['HIRE_DATE']).dt.year).sum()

[문제128] 입사요일별 인원수를 출력하세요.(한글요일로 출력)
import collections
emp['EMPLOYEE_ID'].groupby(pd.to_datetime(emp['HIRE_DATE']).dt.weekday).count()
week = emp['EMPLOYEE_ID'].groupby(pd.to_datetime(emp['HIRE_DATE']).dt.weekday).count()
week.index = ['월','화','수','목','금','토','일']
week

[문제129] 년도, 분기별 급여의 총액을 출력해주세요.
emp.SALARY.groupby([pd.to_datetime(emp['HIRE_DATE']).dt.year,pd.to_datetime(emp['HIRE_DATE']).dt.quarter]).sum()

x = emp.groupby([pd.to_datetime(emp['HIRE_DATE']).dt.year,
             pd.to_datetime(emp['HIRE_DATE']).dt.quarter])['SALARY'].sum()
result = x.unstack()
result.stack()

result.columns = ['1분기','2분기','3분기','4분기']
result.index.name = '년도'
result.fillna(0,inplace=True)
result.stack()

pd.pivot_table(data=emp,
               index=pd.to_datetime(emp['HIRE_DATE']).dt.year,
               columns=pd.to_datetime(emp['HIRE_DATE']).dt.quarter,
               values='SALARY',
               aggfunc='sum')

pd.pivot_table(data=emp,
               index=pd.to_datetime(emp['HIRE_DATE']).dt.year,
               columns=pd.to_datetime(emp['HIRE_DATE']).dt.quarter,
               values='SALARY',
               aggfunc=['sum','mean'])

[문제130] 년도, JOB_ID별 입사한 인원수를 출력해주Tㅔ요.
pd.pivot_table(data=emp,
               index=pd.to_datetime(emp['HIRE_DATE']).dt.year,
               columns=emp.JOB_ID,
               values='JOB_ID',
               aggfunc='count')

x = emp.groupby([pd.to_datetime(emp['HIRE_DATE']).dt.year,'JOB_ID'])['JOB_ID'].count()
result = x.unstack()
result.fillna(0,inplace=True)
result
result.stack()

x = emp.groupby([pd.to_datetime(emp['HIRE_DATE']).dt.year,'JOB_ID'])['EMPLOYEE_ID'].count()
x.index
x.reset_index()
x.reset_index(level='HIRE_DATE') # level 옵션은 hiredate를 열로하겠다?
x.index = pd.MultiIndex.from_tuples([(2001,'AD_VP'),(),()]) # MultiIndex.from_tuples ?

pd.pivot_table(data=emp,
               index=pd.to_datetime(emp['HIRE_DATE']).dt.year,
               columns='JOB_ID',
               values='EMPLOYEE_ID',
               aggfunc='count')

[문제131] 근무일수가 가장 많은 10위까지 직원들의 EMPLOYEE_ID, LAST_NAME, DEPARTMENT_NAME, 근무일수를 출력해주세요.
dept = pd.read_csv('c:/data/departments.csv')
date = pd.Timestamp.now() - pd.to_datetime(emp['HIRE_DATE']) # 근무일수 
date.days # 오류
date[0].days # 하나씩 인덱스들의 days는 구할 수 있음
x = date.rank(ascending=False,method='dense') # 순위
x[x<=10] # 근무일수가 가장많은 10위까지 순위

emp['근무일수'] = (pd.Timestamp.now() - pd.to_datetime(emp['HIRE_DATE'])).dt.days
e = emp[emp['근무일수'].rank(ascending=False,method='dense')<=10][['EMPLOYEE_ID','LAST_NAME','DEPARTMENT_ID','근무일수']]
pd.merge(e,dept,on='DEPARTMENT_ID',how='inner')[['EMPLOYEE_ID','LAST_NAME','DEPARTMENT_NAME','근무일수']]

# 강사님
x =DataFrame({'사번': emp.EMPLOYEE_ID,
           '이름' : emp.LAST_NAME,
           '부서코드' : emp.DEPARTMENT_ID,
           '근무일수' : (pd.Timestamp.now() - pd.to_datetime(emp['HIRE_DATE'])).dt.days})

x['순위'] = x['근무일수'].rank(ascending=False,method='dense').astype(int)
top_10 = x[x['순위'] <= 10].sort_values(by='순위')
pd.merge(top_10,dept,left_on='부서코드',right_on='DEPARTMENT_ID')[['사번','이름','근무일수','DEPARTMENT_NAME','순위']]


emp.info()
emp.HIRE_DATE = pd.to_datetime(emp.HIRE_DATE,format='%Y-%m-%d') # 날짜형으로 바꿔놓기
emp.info()

x =DataFrame({'사번': emp.EMPLOYEE_ID,
           '이름' : emp.LAST_NAME,
           '부서코드' : emp.DEPARTMENT_ID,
           '근무일수' : (pd.Timestamp.now() - emp['HIRE_DATE']).dt.days})

emp = pd.read_csv('c:/data/employees.csv',parse_dates=['HIRE_DATE']) # 파일을 읽을 때 hire_date컬럼을 date타입형식으로 읽겠다.
emp.info()

emp = pd.read_csv('c:/data/employees.csv',
                  parse_dates=['HIRE_DATE'],
                  date_parser=lambda arg : pd.to_datetime(arg,format='%Y-%m-%d')) # 날짜 포멧을 지정 %Y-%m-%d
emp.info()

##########################################

★ matplotlib
- 시각화 라이브러리

1. pie chart
- 범주형 자료에 대한 그래프를 그릴때 사용
- 원을 그린 후 그 원에 각 계급의 상대도수에 대한 면적 또는 부분으로 나눈다.

선호도    도수    상대도수    각도
---------------------------------
좋다      45       0.45      0.45 * 360 = 162
보통      25       0.25
싫다      20       0.2
무응답    10       0.1

import matplotlib.pylab as plt 
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/windows/fonts/HMKMMAG.TTF").get_name()
rc('font',family=font_name)

labels = ['좋다','보통','싫다','무응답']
frequency = [45,25,20,10]
# matplotlib은 데이터프레임으로 구성되야지 표현할 수 있다.

survey = DataFrame({'frequency':frequency},index=labels)
survey.frequency.plot(kind='pie')
survey.frequency.plot(kind='pie',
                      figsize=(7,5),  #그래프 사이즈(가로, 세로)
                      autopct='%.1f%%', # 퍼센트 값 표현
                      startangle=40, # 시작각도
                      colors=['skyblue','red','green','bisque'], # 색
                      title='고객만족도조사', # 제목
                      legend=True, # 범례
                      fontsize=10 # 글씨크기
                      )

survey.frequency.plot.pie()
plt.pie(survey.frequency,
        shadow=True, #그림자
        autopct='%.1f%%',
        #explode=(0.1,0.1,0,0), # 조각내기
        wedgeprops={"linewidth": 1, "edgecolor": "white"}
        )
plt.title("고객만족도조사",size=10)
#plt.legend(labels=survey.index,loc='best') # loc='best 최적의 위치를 알아서 찾아서 표현
plt.legend(labels=survey.index,loc='lower center', ncol=4) # ncol=4 가로형태로 표현

plt.pie(survey.frequency,
        shadow=True, #그림자
        autopct='%.1f%%',
        #explode=(0.1,0.1,0,0), # 조각내기
        wedgeprops={"width":0.7,"linewidth": 1, "edgecolor": "white"} # width는 가운데 구멍 뚫어줌, 테두리 선
        )
plt.show()

[문제132] blood.csv 파일을 읽어 들여서 도수분포표를 작성하고 pi chart 생성해주세요.
blood = pd.read_csv('c:/data/blood.csv')

# 도수분포표 생성
#blood.groupby(['BLOODTYPE','GENDER']).count().unstack()
x = blood.groupby('BLOODTYPE')['GENDER'].count()
x = DataFrame(x)
x['cnt'] = [ i/x.GENDER.sum() for i in x.GENDER] 
x

# 차트생성
x.cnt.plot.pie()
x.cnt.plot(kind='pie',
           autopct='%.1f%%',
           legend=True,
           startangle=90,
           colors=['black','orange','yellow','purple'],
           title='혈액형조사',
           fontsize=12)

plt.pie(x.cnt,
        shadow=True,
        autopct='%.1f%%',
        explode=(0.1,0.1,0.1,0.1),
        wedgeprops={"width":0.7})

#강사님

x = blood.BLOODTYPE.value_counts()
plt.pie(x,
        shadow=True,
        autopct='%.1f%%',
        colors=['black','orange','yellow','purple'],
        explode=(0.1,0.1,0.1,0.1),
        textprops={'fontsize':10,'color':'white'},
        wedgeprops={"width":0.7,"linewidth": 1, "edgecolor": "white"})
plt.title('혈액형현황',fontsize=10)
plt.legend(labels=x.index,loc='lower center',ncol=4)
plt.show()

[문제133] 요일별 입사인원수를 pie chart 생성해주세요
x = emp.groupby(emp['HIRE_DATE'].dt.weekday)['EMPLOYEE_ID'].count()
x.index = ['월','화','수','목','금','토','일']
x
plt.pie(x,
        shadow=True,
        autopct='%.f%%',
        explode=(0.1,0.1,0.1,0.1,0.1,0.1,0.1),
        textprops={'fontsize':10,'color':'blue'},
        wedgeprops={'width':0.8,'linewidth': 5, 'edgecolor':'black'},
        #frame=True,
        labels=week.index)
plt.title('요일별 입사인원수',fontsize=13)
plt.legend(labels=x.index,loc='lower right')


2. 막대그래프(bar chart)
- 각 범주에서 도수의 크기를 막대높이로 표현하는 그래프

x = [1,2,3]
y = [90,79,65]
plt.bar(x,y,color=['orange','green','blue'])
plt.title('과목별 점수 현황',size= 10)
plt.xlabel('과목',size=7)
plt.ylabel('점수',size=7)
plt.xticks(x,['SQL','R','PYTHON'])

[문제134] 부서별 입사인원수를 막대그래프로 시각화해주세요. 단 소속부서가 없는 사원의 인원수도 출력해주세요.
emp.groupby('DEPARTMENT_ID')['EMPLOYEE_ID'].count()
x =emp.EMPLOYEE_ID.groupby(emp.DEPARTMENT_ID).count()

dept_cnt = emp.DEPARTMENT_ID.fillna(999).value_counts()
dept_cnt.sort_index(inplace=True)
dept_cnt = dept_cnt.reset_index()
dept_cnt.columns = ['부서','도수']
dept_cnt['부서'] = dept_cnt.부서.astype(int)
dept_cnt['도수'] / dept_cnt['도수'].sum()

dept_cnt.loc[dept_cnt['부서']==999,'부서'] = '부서x'
dept_cnt

# 수직 막대그래프
plt.bar(dept_cnt.index,dept_cnt.도수)
plt.title('부서 인원수 현황',size=10)
plt.xlabel('부서',size=8)
plt.ylabel('인원수',size=8)
plt.xticks(dept_cnt.index,dept_cnt.부서)

dept_cnt.도수.plot(kind='bar')
plt.title('부서 인원수 현황',size=10)
plt.xlabel('부서',size=8)
plt.ylabel('인원수',size=8)
plt.xticks(dept_cnt.index,dept_cnt.부서,fontsize=8,color='red',rotation=0)
plt.yticks(range(0,55,5))

# 수평 막대그래프
plt.barh(dept_cnt.index,dept_cnt.도수)
plt.yticks(dept_cnt.index,dept_cnt.부서)
plt.title('부서 인원수 현황',size=10)
plt.ylabel('부서',size=8)
plt.xlabel('인원수',size=8)
plt.xticks(range(0,55,5))

dept_cnt.도수.plot(kind='barh')
plt.yticks(dept_cnt.index,dept_cnt.부서)
plt.title('부서 인원수 현황',size=10)
plt.ylabel('부서',size=8)
plt.xlabel('인원수',size=8)
plt.xticks(range(0,55,5))

# crosstab -> sequence하게 값이 만들어짐,
x_cross = pd.crosstab(emp.DEPARTMENT_ID.fillna(999),columns='도수')
x_cross.plot(kind='bar')
x_cross.plot(kind='barh')

data = {'홍길동':[15,13,11],
        '제임스':[13,14,15],
        '하든':[10,9,12]}
df = DataFrame(data,index=[2020,2021,2022])
df
df.홍길동.plot(kind='bar')
df.홍길동.plot(kind='barh')

# 그룹형 막대그래프
df.plot(kind='bar')
df.plot(kind='barh')

# 스택형 막대그래프
df.plot(kind='bar',stacked=True)
df.plot(kind='barh',stacked=True)


[문제135] fruits_sales.csv 의 데이터를 이용해서 과일의 년도별 수량을 막대그래프로 시각화 해주세요.
sales = pd.read_csv('c:/data/fruits_sales.csv')

x = sales.groupby(['year','name'])['qty'].sum().unstack()
x.plot(kind='bar')
x.plot(kind='barh')
x.plot(kind='bar',stacked=True)
x.plot(kind='barh',stacked=True)

sales_df = pd.pivot_table(data=sales,
                          index='year',
                          columns='name',
                          values='qty',
                          aggfunc='sum')
sales_df.plot(kind'bar')


x = [1,2,3]
y = [90,80,70]

font = {'color':'darkred','size':10}
box = dict(boxstyle='round',edgecolor='white',facecolor='blue')

plt.bar(x,y)
plt.text(1,91,90,fontdict=font, bbox=box) 
plt.text(3,75,'열심히 공부하자',fontdict=font, bbox=box,rotation=45)

[문제136] 요일별 입사 인원수를 수직형 막대그래프로 시각화 해주세요. 빈도수도 출력해주세요.
emp.groupby('HIRE_DATE')['EMPLOYEE_ID'].count()
x = emp.EMPLOYEE_ID.groupby(emp.HIRE_DATE.dt.weekday).count()
x.index = ['월','화','수','목','금','토','일']
plt.bar(x.index,x)
for i in range(0,len(x)):
    plt.text(i,x[i],x[i])

plt.bar(x.index,x)
[plt.text(i,x[i],x[i]) for i in range(0,len(x))]

week = emp.EMPLOYEE_ID.groupby(emp.HIRE_DATE.dt.weekday).count()
plt.bar(week.index,week)
plt.xticks(week.index,['원','화','수','목','금','토','일'])
for i in week.index:
    plt.text(i-0.1,week[i]+0.2,week[i])

